In [8]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 25 22:07:41 2017

@author: I327247
"""

import spotipy
import spotipy.util as util
import os
import json
import sys
import pandas as pd

#Application (Muse) credentials
os.environ["SPOTIPY_CLIENT_ID"] = 'd79b7361b6e74f6d935c68d1cdb32502'
os.environ["SPOTIPY_CLIENT_SECRET"] = '4b5df00b9f1a4dfd89b1a535242ef724'
os.environ["SPOTIPY_REDIRECT_URI"] = 'http://localhost:3030/callback'

#Dump function for a better display of dictionnaries objects
def dumpclean(obj):
    print(json.dumps(obj, indent = 10))
    
#Print the track
def show_track(track):
        print("%s %32.32s %s" % ('Track', track['artists'][0]['name'], track['name']))


def main(sp):
    """
        Main Function : Take the Spotify context as parameter 
    """
    #ranges = ['short_term', 'medium_term', 'long_term']
    time_range = 'short_term'

    tracks_id = []
    
    #Dataframe containing Tracks overall description
    track_df = pd.DataFrame(columns=['id', 'name', 'artist', 'popularity', 'duration_ms'])
    
    #Dataframe containing features of the tracks
    feat_columns=['id', 'danceability', 'energy', 'key', 'loudness', 'mode',
                'speechiness', 'acousticness', 'instrumentalness',
                'liveness', 'valence', 'tempo', 'duration_ms', 
                'time_signature']
    feature_df = pd.DataFrame(columns=feat_columns)

    print("range:", time_range)
    results = sp.current_user_top_tracks(time_range=time_range)
    #dumpclean(results)
    while results['next']:
        results = sp.next(results)
        for i, item in enumerate(results['items']) :
            track = sp.track(item['uri'])
            tracks_id.append(item['id'])
            
            #Add track to the tracks dataframe
            row = [item['id'], track['name'], track['artists'][0]['name'],
                   track['popularity'], track['duration_ms']]
            track_df.loc[len(track_df)] = row
            show_track(track)
    
    #Get the tracks features
    features_results = sp.audio_features(tracks_id)
    
    for track_features in features_results :
        solo_feat_df = pd.DataFrame(track_features, index=[0])
        solo_feat_df = solo_feat_df[feat_columns]
        feature_df = feature_df.append(solo_feat_df, ignore_index=True)
        
    print(feature_df.describe())
    
    #Dump the dataframe
    track_df.to_csv('tracks.csv', index=False, encoding='utf-8')
    
    feature_df.to_csv('tracks_features.csv', index=False, encoding='utf-8')
    #Join the two dataframes

if __name__ == '__main__':
    username = 'marshallerikson'

    #Let's start with the public scope and we'll refine it when needed
    scope = 'user-top-read'
    
    #Create the authentication token.
    #Also use the Muse credentials to connect to the right app
    token = util.prompt_for_user_token('marshallerikson', scope=scope)
    
    if token :
        print('Authentication successfull !!!')
        sp = spotipy.Spotify(auth=token)
        print(sp.current_user())
        main(sp)        
    else : 
        print('Authentication failed. Can\'t get token for', username)
        sys.exit()

Authentication successfull !!!
{'uri': 'spotify:user:cyrahl12', 'id': 'cyrahl12', 'images': [], 'followers': {'total': 0, 'href': None}, 'display_name': None, 'href': 'https://api.spotify.com/v1/users/cyrahl12', 'external_urls': {'spotify': 'https://open.spotify.com/user/cyrahl12'}, 'type': 'user'}
range: short_term
Track                         P-Square Shekini
Track                              MHD Ma vie
Track                       Yemi Alade Temperature (feat. Dil)
Track                        Kedjevara Remue la bouteille
Track                            Tekno Duro
Track                         P-Square Collabo
Track                             Zaho Laissez-les kouma (feat. MHD)
Track                          Black M Kirikou
Track                              MHD Afro Trap Pt. 4 (Fais le mouv)
Track                             Zedd Ignite - 2016 League Of Legends World Championship
Track                          Bana C4 Na Lela Yo
Track                         Banky W. All I Want I